In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
#LDA -LDA is a supervised dimensionality reduction method, meaning it uses the target labels (y)
def pcared(x_train,y_train,x_test,n):
    feature_names = [f'Feature{i+1}' for i in range(x_train.shape[1])]
    from sklearn.decomposition import KernelPCA
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    lda = LDA(n_components = n)
    ldafit=lda.fit_transform(x_train,y_train)
    ldafea=lda.transform(x_test)
    principalDf = pd.DataFrame(data = ldafit, columns = ['principal component 1'])
    print(principalDf)
    #pca = PCA(n_components = n)
    #pcafit=pca.fit_transform(x_train)
    #pcafea=pca.transform(x_test)
    #principalDf = pd.DataFrame(data = pcafit, columns = ['principal component 1', 'principal component 2','principal component 3'])
    #print(principalDf)
    explained_variance = lda.explained_variance_ratio_
    print(explained_variance)
    return ldafit,ldafea

In [3]:
def standardscaler(indepX,depY):
    #Train and Test set split
    x_train,x_test,y_train,y_test=train_test_split(indepX,depY,test_size=0.30,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

In [4]:
def cmreport(classifier,x_test):
    y_pred=classifier.predict(x_test)
    cm=confusion_matrix(y_test,y_pred)
    accuracy=accuracy_score(y_test,y_pred)
    clr=classification_report(y_test,y_pred)
    return classifier,cm,accuracy,clr,x_test,y_test

In [5]:
def Random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    RFC=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    RFC.fit(x_train,y_train)
    classifier,cm,accuracy,clr,x_test,y_test=cmreport(RFC,x_test)
    return classifier,cm,accuracy,clr,x_test,y_test

In [6]:
def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    DT=DecisionTreeClassifier(criterion = 'entropy',random_state = 0)
    DT.fit(x_train,y_train)
    classifier,cm,accuracy,clr,x_test,y_test=cmreport(DT,x_test)
    return classifier,cm,accuracy,clr,x_test,y_test

In [7]:
def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    kn=KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    kn.fit(x_train,y_train)
    classifier,cm,accuracy,clr,x_test,y_test=cmreport(kn,x_test)
    return classifier,cm,accuracy,clr,x_test,y_test

In [8]:
def XGB(x_train,y_train,x_test):
    from xgboost import XGBClassifier
    bst=XGBClassifier(learning_rate=0.3,max_depth=5,n_estimators=100)
    bst.fit(x_train,y_train)
    classifier,cm,accuracy,clr,x_test,y_test=cmreport(bst,x_test)
    return classifier,cm,accuracy,clr,x_test,y_test

In [9]:
def QDA(x_train,y_train,x_test):
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(x_train,y_train)
    classifier,cm,accuracy,clr,x_test,y_test=cmreport(qda,x_test)
    return classifier,cm,accuracy,clr,x_test,y_test

In [10]:
def pcaconfusion(accran,accdeci,accknn,accxgb,accqda):
    df=pd.DataFrame(index=['PCA'],columns=['Random','Decision','KNN','XGBoost','QuadraticDiscriminantAnalysis(QDA)'])
    for number,idex in enumerate(df.index):
        df.loc[idex, 'Random'] = accran[number]
        df.loc[idex, 'Decision'] = accdeci[number]
        df.loc[idex, 'KNN'] = accknn[number]
        df.loc[idex, 'XGBoost'] = accxgb[number]
        df.loc[idex, 'QuadraticDiscriminantAnalysis(QDA)'] = accqda[number]
        return df

In [11]:
dataset=pd.read_csv("QualityPrediction.csv",index_col=None)
dataset.isnull().sum()

aluminium      0
ammonia        0
arsenic        0
barium         0
cadmium        0
chloramine     0
chromium       0
copper         0
flouride       0
bacteria       0
viruses        0
lead           0
nitrates       0
nitrites       0
mercury        0
perchlorate    0
radium         0
selenium       0
silver         0
uranium        0
is_safe        0
dtype: int64

In [12]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [13]:
indepX=dataset.drop('is_safe_Unsafe',axis=1)
depY=dataset['is_safe_Unsafe']
x_train,x_test,y_train,y_test=standardscaler(indepX,depY)

In [14]:
x_train,x_test=pcared(x_train,y_train,x_test,1)  

      principal component 1
0                  0.841709
1                  3.442131
2                 -1.183711
3                 -0.091853
4                  0.878241
...                     ...
5592               0.045247
5593              -0.312387
5594               0.315767
5595               1.102328
5596               1.386186

[5597 rows x 1 columns]
[1.]


In [15]:
accran=[]
accdeci=[]
accknn=[]
accxgb=[]
accqda=[]

In [16]:
classifier,cm,accuracy,clr,x_test,y_test=Random(x_train,y_train,x_test)
accran.append(accuracy)
classifier,cm,accuracy,clr,x_test,y_test=Decision(x_train,y_train,x_test)
accdeci.append(accuracy)
classifier,cm,accuracy,clr,x_test,y_test=knn(x_train,y_train,x_test)
accknn.append(accuracy)
classifier,cm,accuracy,clr,x_test,y_test=XGB(x_train,y_train,x_test)
accxgb.append(accuracy)
classifier,cm,accuracy,clr,x_test,y_test=QDA(x_train,y_train,x_test)
accqda.append(accuracy)

In [17]:
result=pcaconfusion(accran,accdeci,accknn,accxgb,accqda)
print(result)

       Random  Decision       KNN   XGBoost QuadraticDiscriminantAnalysis(QDA)
PCA  0.848687  0.847853  0.879116  0.884952                           0.892872
